In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# BERT

## Dependencies

In [ ]:
!pip install tensorflow
!pip install tensorflow_hub
!pip install tensorflow_text

## Loading and Cleaning Dataset

In [3]:
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/data_augmented_2.csv')
df.head(5)

,Unnamed: 0,request,response,pages,name
0,0,"can you print the document mapa.pdf please, th...",1,250,mapa.pdf
1,1,"fast, print me the document named result.pdf t...",1,5000,result.pdf
2,2,i need the document hayhya.pdf printed by tomo...,1,1000,hahya.pdf
3,3,"print me doc1.pdf, it has 50pages",1,50,doc1.pdf
4,4,"print calendar.pdf, it has 65 pages",1,65,calendar.pdf


## Train Test Split

In [4]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df['request'], df['response'], test_size = 0.3, 
                                                   shuffle = True, stratify = df['response'], random_state = 7)

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, 
                                                  shuffle = True, stratify = y_train, random_state = 7)

print(f'\nShape checks:\nX_train: {X_train.shape} X_test: {X_test.shape} X_val: {X_val.shape} \ny_train: {y_train.shape} y_test: {y_test.shape} y_val: {y_val.shape}')


Shape checks:
X_train: (26276,) X_test: (15015,) X_val: (8759,) 
y_train: (26276,) y_test: (15015,) y_val: (8759,)


## Embedding

In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text

In [ ]:
bert_preprocess = hub.KerasLayer('https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3')
bert_encoder = hub.KerasLayer('https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4')

## Model

In [ ]:
# Bert Layers
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name="text")
preprocessed_text = bert_preprocess(text_input)
outputs = bert_encoder(preprocessed_text)
# Neural nerwork layers
l = tf.keras.layers.Dropout(0.1, name="dropout")(outputs['pooled_output'])
l = tf.keras.layers.Dense(1, activation="sigmoid", name="output")(l)
# the inputs and outputs to construct a final model
model = tf.keras.Model(inputs=[text_input], outputs = [l])

## Model summary

In [ ]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 text (InputLayer)              [(None,)]            0           []                               
                                                                                                  
 keras_layer (KerasLayer)       {'input_type_ids':   0           ['text[0][0]']                   
                                (None, 128),                                                      
                                 'input_word_ids':                                                
                                (None, 128),                                                      
                                 'input_mask': (Non                                               
                                e, 128)}                                                      

## Compile model

In [ ]:
from keras import backend as K

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['acc',f1_m,precision_m, recall_m])

## Fit model

In [ ]:
hist = model.fit(X_train, y_train, epochs=5, validation_data=(X_val, y_val))

Epoch 1/5
822/822 [==============================] - 351s 423ms/step - loss: 0.4497 - acc: 0.8000 - f1_m: 0.8031 - precision_m: 0.7759 - recall_m: 0.8529 - val_loss: 0.3554 - val_acc: 0.8829 - val_f1_m: 0.8899 - val_precision_m: 0.8229 - val_recall_m: 0.9753
Epoch 2/5
822/822 [==============================] - 347s 423ms/step - loss: 0.3371 - acc: 0.8731 - f1_m: 0.8737 - precision_m: 0.8506 - recall_m: 0.9082 - val_loss: 0.2907 - val_acc: 0.9040 - val_f1_m: 0.9053 - val_precision_m: 0.8725 - val_recall_m: 0.9462
Epoch 3/5
822/822 [==============================] - 347s 422ms/step - loss: 0.2976 - acc: 0.8864 - f1_m: 0.8867 - precision_m: 0.8671 - recall_m: 0.9167 - val_loss: 0.2694 - val_acc: 0.9052 - val_f1_m: 0.8987 - val_precision_m: 0.9404 - val_recall_m: 0.8663
Epoch 4/5
822/822 [==============================] - 346s 421ms/step - loss: 0.2707 - acc: 0.8967 - f1_m: 0.8970 - precision_m: 0.8787 - recall_m: 0.9238 - val_loss: 0.2667 - val_acc: 0.8928 - val_f1_m: 0.8827 - val_precisi

## Evaluate model

In [ ]:
model.evaluate(X_test, y_test)

470/470 [==============================] - 153s 325ms/step - loss: 0.2056 - acc: 0.9479 - f1_m: 0.9467 - precision_m: 0.9350 - recall_m: 0.9624


[0.2056431621313095,
 0.9478521347045898,
 0.9466791749000549,
 0.9350407123565674,
 0.9623993039131165]

## Saving Model

In [ ]:
PATH = "/content/drive/MyDrive/model"
model.save(PATH)